# 量子エラーの訂正

<!--
Unit 4: Foundations of Quantum Error Correction
Lesson 13: Correcting Quantum Errors
-->

<iframe src="https://video.ibm.com/embed/recorded/134082557?showtitle=false" loading="lazy" width="100%" height="450" frameborder="0" allow="accelerometer; autoplay; clipboard-write; encrypted-media; gyroscope; picture-in-picture; web-share" allowfullscreen>
</iframe>

このレッスンの [YouTube ビデオ](https://youtu.be/OoQSdcKAIZc)を別のウィンドウで開きます。

## はじめに

量子コンピューティングは、効率的な古典アルゴリズムが知られていないか存在しない可能性のある計算課題に対して、効率的な解法を提供する可能性を秘めています。
しかし、私たちが将来的に期待するような大規模な量子計算を確実に実現するためには、非常に大きな課題を克服する必要があります。

問題の核心は、量子情報が極めて壊れやすいことです――文字通り、それを「見る」だけで情報を破壊してしまう可能性があります。
このため、量子コンピュータが正しく動作するには、記憶している量子情報を周囲の環境から極限まで隔離しなければなりません。
しかし同時に、量子コンピューターは、適切な初期化、正確で信頼性のあるユニタリー操作、そして計算結果を得るための測定を含め、量子情報に対して非常に精密な制御を提供する必要があります。

これらの要件の間には明らかに緊張関係があり、量子コンピューティングの黎明期には、量子情報の脆弱性や、誤りや環境ノイズへの影響を受けやすい性質が、最終的に量子コンピューティングを不可能にするのではないかと考える人もいました。
今日では、大規模な量子コンピューターを正確かつ信頼性のあるものとして構築することが非常に困難な課題であることは疑いようがありません。
しかし、この挑戦において私たちを支えてくれる重要なツールがあり、それこそが、大規模な量子コンピューティングが将来的に実現可能であると、多くの専門家が楽観視する理由となっています。そのツールとは、量子誤り訂正です。

今後4回のレッスンでは、量子誤り訂正について、基礎に焦点を当てながら学んでいきます。本レッスンでは、量子誤り訂正の最初の概観を示し、最初に発見された量子誤り訂正符号である *9量子ビットのショアの符号（9-qubit Shor code）* を紹介するとともに、量子誤り訂正の基礎概念の一つである *誤りの離散化（discretization of errors）* についても議論します。

## 古典的な繰り返し符号

このレッスンでは、 *古典的な繰り返し符号* から始めます。これは、9量子ビットのショアの符号の基礎となるものです。

### 符号化と復号化

繰り返し符号は、誤り訂正符号の中でも非常に基本的な例です。
この考え方は、各ビットを一定回数繰り返して符号化することで、誤りからビットを保護できるというものです。
特に、まずは 3ビットの繰り返し符号 を考えてみましょう。
ここでは、1つのビットを3回繰り返すことで符号化します。つまり、$0$ は $000$ に、$1$ は $111$ に符号化されます。

$$
\begin{aligned}
0 & \mapsto 000\\
1 & \mapsto 111
\end{aligned}
$$

何も問題が起こらなければ、符号化されたデータから元のビットを明確に区別できます。
しかし、1つのビットが誤って反転してしまい、$0$ が $1$ に変わったり、$1$ が $0$ に変わったりした場合でも、3ビットのうち どちらの値が2回現れるか を確認することで、元のビットを推測できます。
これはつまり、多数決を取る(つまり最も頻繁に現れるバイナリー値を取る)ことで *復号（デコード）* できるということです。

$$
a b c \mapsto \operatorname{majority}(a,b,c)
$$

もちろん、2つまたは3つのビットが誤って反転してしまうと、復号がうまくいかず、誤ったビットが得られてしまいます。しかし、1つまでの誤りであれば正しく復号できることがわかります。
これは誤り訂正符号の典型的な特性であり、ある程度の誤りを訂正できるものの、誤りが多すぎると訂正が難しくなるという特徴を持っています。

### 二元対称チャネル

繰り返し符号を用いることで誤りの発生確率を減らせる具体例として、単一のビットを仮想的な受信者に伝達したい状況を考えてみましょう。
私たちは、 *二元対称チャネル（binary symmetric channel）* と呼ばれる通信経路を通じてビットを送信できるとします。
このチャネルでは、送信されたビットが *独立して* 確率 $p$ で反転する可能性があります。
つまり、確率 $1-p$ で受信者は送信されたビットをそのまま受け取りますが、確率 $p$ で誤りが発生し、反対の値が届いてしまいます。

もし3ビットの繰り返し符号を使用せず、そのままビットを送信する場合、受信者が誤ったビットを受け取る確率は $p$ です。
一方で、送信したいビットをまず3ビットの繰り返し符号で符号化し、それぞれのビットをチャネルを通じて送る場合、3つのビットのそれぞれが独立して確率 $p$ で反転することになります。
ビットが反転する可能性は、1ビット送る場合よりも高まります（なぜなら、3つのビットのどれもが反転する可能性があるため）が、最大1つのビットが反転するだけであれば、受信者は正しく復号できます。
したがって、誤りが残るのは、2つ以上のビットが反転してしまった場合のみです。

2つのビットが誤って反転する確率は $3p^2(1-p)$ です。これは、反転しないビットを3通りのうち1つ選ぶ確率 $p^2(1-p)$ を考えた結果です。
また、3つすべてのビットが反転する確率は $p^3$となります。よって、2ビットまたは3ビットが反転する合計確率は次のようになります。

$$
3 p^2 (1 - p) + p^3 = 3 p^2 - 2 p^3.
$$

もし $p$ が 1/2 より小さい 場合、この式の値は $p$ よりも小さくなるため、受信者が誤ったビットを受け取る確率は減少します。
つまり、この符号には完全ではないものの、誤りの発生確率を *低減する* 効果があるのです。
(一方で、$p$ が 1/2 より大きい 場合には、この符号を使うことで誤りの発生確率が逆に *増加* してしまいます。)

![Error probability graph for the 3 bit repetition code for a binary symmetric channel](https://learning-api.quantum.ibm.com/assets/5eefd3f1-a9e9-45e9-a622-dc85e84d9dce?format=auto&quality=80)

## 量子ビットに対する繰り返し符号

3ビットの繰り返し符号は古典的な誤り訂正符号ですが、これを *量子ビット* の誤りを防ぐために使用しようとするとどうなるかを考えてみましょう。
これから見るように、この方法は量子誤り訂正符号としてはあまり優れていません。むしろ、特定の種類の誤りを発生しやすくしてしまう場合があります。
しかし、この符号はショアの符号への第一歩となるものであり、教育的な観点からも役立ちます。

### 符号化

ここで、量子ビットに対して3ビットの繰り返し符号を使用するとは、 *標準基底状態* を3回繰り返して符号化することを意味します。
つまり、1量子ビットの状態ベクトルは、次のように符号化されます。

$$
\alpha \vert 0\rangle + \beta \vert 1\rangle \mapsto \alpha \vert 000\rangle + \beta \vert 111\rangle
$$

この符号化は、2つの補助的な量子ビットを初期化し、2つの制御NOT（CNOT）ゲートを適用することで容易に実装 できます。

![Encoding circuit for the 3-bit repetition code](https://learning-api.quantum.ibm.com/assets/6638f32f-7202-4628-a35f-5cd217a00026?format=auto&quality=80)

特に重要なのは、この符号化は単に量子状態を3回繰り返しているわけではない という点です。
つまり、量子ビットの状態ベクトルを$\vert\psi\rangle \mapsto \vert\psi\rangle\vert\psi\rangle\vert\psi\rangle$のように符号化しているわけではありません。
このような符号化は、クローン不可能定理（no-cloning theorem）により、未知の量子状態 $\vert\psi\rangle$ に対して実装することができません。

### ビット反転エラーの検出

ここで、符号化を行った後にエラーが発生する場合を考えてみましょう。
具体的には、$X$ ゲート、つまりビット反転が量子ビットの1つに作用するとします。
例えば、中央の量子ビットがビット反転を受けた場合、3量子ビットの状態は次のように変化します。

$$
\alpha \vert 010\rangle + \beta \vert 101\rangle.
$$

もちろん、発生しうるエラーはこの種類だけではありません。また、エラーが完全なユニタリー演算の形を取る と仮定するのが妥当なのかも疑問に思うかもしれません。
これらの問題については、レッスンの最後のセクションで詳しく扱いますが、今のところ、この形式のエラーが考えうる1つのケース（しかも非常に重要なケース）であると捉えておきましょう。

上の数式を見ると、各ケットの中で中央のビットだけが異なっている ことが明らかです。しかし、もし私たちがこの3量子ビットを持っているものの、その状態を知らない場合はどうなるでしょうか？
ビット反転が発生した可能性があると考えたとき、それを確認する1つの方法は標準基底で測定することです。この場合、測定の結果は $010$ または $101$ となり、それぞれの確率は $\vert\alpha\vert^2$ および $\vert\beta\vert^2$ となります。
この結果から、中央のビットが反転した ことは分かりますが、残念ながら元の量子状態 $\alpha\vert 0\rangle + \beta \vert 1\rangle$ は失われてしまいます。私たちが保護しようとしているのはこの状態であるため、標準基底での測定は望ましい選択肢とは言えません。

代わりに、以下の量子回路 を使用することで、この問題を回避できます。この回路では、符号化された状態を上側の3つの量子ビットに入力します。
そして、この回路は上側の2つの量子ビット、および下側の2つの量子ビットの標準基底状態の *パリティ（偶奇）* を非破壊的に測定します。

![Error detection circuit for the 3-bit repetition code (no errors)](https://learning-api.quantum.ibm.com/assets/52c395f5-9289-4895-88c4-5586de98700c?format=auto&quality=80)

最大で1つのビットが反転するという仮定のもとでは、測定結果からビット反転が発生した場所（または発生していないこと）を容易に特定できます。
特に、以下の4つの回路図が示すように、測定結果が $00$ であればビット反転は発生していないことを意味し、その他の3つの測定結果は、それぞれどの量子ビットがビット反転を受けたかを示します。

![Error detection circuit for the 3-bit repetition code (no errors)](https://learning-api.quantum.ibm.com/assets/443a6c65-9f03-4878-a96e-2762b9a4d35a?format=auto&quality=80)
![Error detection circuit for the 3-bit repetition code (error on 2)](https://learning-api.quantum.ibm.com/assets/364dfb67-29d7-45b2-8348-64ca97fc21fe?format=auto&quality=80)
![Error detection circuit for the 3-bit repetition code (error on 1)](https://learning-api.quantum.ibm.com/assets/0b8e5bb9-8e7b-4815-9a36-f50ef2f39de0?format=auto&quality=80)
![Error detection circuit for the 3-bit repetition code (error on 0)](https://learning-api.quantum.ibm.com/assets/a12594c9-2537-4024-9384-921c9d0cf22c?format=auto&quality=80)

重要な点として、上側の3つの量子ビットの状態は、どの場合でも崩壊しません。これにより、ビット反転の誤りが発生していた場合でも、単にもう一度同じビット反転を適用することで修正することができます。  
以下の表に、最大で1つのビット反転が発生した場合の状態の変化、測定結果（誤り訂正の文脈では *シンドローム* と呼ばれる）、および元の符号化状態に戻すために必要な修正をまとめます。

| 状態 | シンドローム | 修正 |
|:-:|:-:|:-:|
| $\alpha\vert 000\rangle + \beta \vert 111\rangle$ | $00$ | $\mathbb{I}\otimes\mathbb{I}\otimes\mathbb{I}$ |
| $\alpha\vert 100\rangle + \beta \vert 011\rangle$ | $10$ | $X\otimes\mathbb{I}\otimes\mathbb{I}$ |
| $\alpha\vert 010\rangle + \beta \vert 101\rangle$ | $11$ | $\mathbb{I}\otimes X\otimes\mathbb{I}$ |
| $\alpha\vert 001\rangle + \beta \vert 110\rangle$ | $01$ | $\mathbb{I}\otimes\mathbb{I}\otimes X$ |

繰り返しになりますが、最大で1つのビット反転が発生する場合のみを考えています。
もし2つまたは3つのビット反転が発生した場合、この方法は正しく機能しません。また、ビット反転以外の誤りの可能性についても、ここではまだ考慮していません。

### 位相反転エラー

量子の場合、ビット反転エラーだけではなく、他の種類のエラーについても考える必要があります。
例えば、 *位相反転エラー（phase-flip error）* もその1つであり、これは $Z$ ゲートによって表されます。
ビット反転エラーと同様に、位相反転エラーも量子ビットに影響を与えるエラーの1つの可能性として考えることができます。
しかし、レッスンの最後のセクションで学ぶ量子誤り訂正符号における *誤りの離散化（discretization of errors）* に関する議論のとおり、ビット反転エラーと位相反転エラーに焦点を当てることは理にかなっています。
特に、ビット反転エラー、位相反転エラー、またはその両方を修正する能力があれば、単一の量子ビットに対する任意の量子エラーを修正できることが保証されます。

しかし、3ビット反復符号は位相反転エラーに対してまったく効果がありません。
例えば、量子ビット状態 $\alpha\vert 0\rangle + \beta\vert 1\rangle$ を3ビット反復符号で符号化した後に、中央の量子ビットに位相反転誤りが発生 したとします。
このとき、状態は次のように変化します。

$$
(\mathbb{I} \otimes Z \otimes \mathbb{I}) ( \alpha \vert 000\rangle + \beta \vert 111\rangle)
= \alpha \vert 000\rangle - \beta \vert 111\rangle,
$$

これは、もともとの量子ビット状態が $\alpha\vert 0\rangle - \beta\vert 1\rangle$ であった場合の符号化結果と全く同じ状態 になります。
実際、符号化された3つの量子ビットのうち、どれか1つに位相反転エラーが発生すると、この同じ影響を受けます。
これは、符号化前の単一量子ビットに位相反転エラーが発生した場合と等価であることを意味します。
したがって、元の量子状態が未知であると仮定すると、エラーが発生したことを検出する手段はありません。
なぜなら、それは単に異なる量子ビット状態の正しい符号化結果だからです。
特に、先ほどの誤り検出回路を状態 $\alpha \vert 000\rangle - \beta \vert 111\rangle$ に適用した場合、測定結果（シンドローム）は必ず $00$ になり、エラーが発生していないと誤って判断されてしまいます。

さらに問題なのは、符号化後の3つの量子ビットはもともと1つだった量子ビットよりも位相反転エラーを受けやすくなっていることです。
例えば、位相反転エラーが各量子ビットに対して独立に確率 $p$ で発生する状況（ビット反転の場合の二元対称チャネルに類似）を考えると、この符号化は小さな$p$ の場合に、デコード後の位相反転エラーの確率をむしろ増加させてしまいます。
具体的には、3つの符号化量子ビットのうち奇数個に位相反転エラーが発生すると、デコード後の元の量子ビットに位相反転エラーが生じることになります。
その確率は $3 p (1 - p)^2 + p^3$ です。
この値は $0<p<1/2$の範囲では $p$ より大きくなるため、この符号化を行うと、位相反転エラーの確率がむしろ増加してしまうことになります。

### 位相反転エラーに対応する修正済み反復符号

これまでに、3ビット反復符号は位相反転エラーを全く検出できないことを確認しました。そのため、この種のエラーに対してはあまり役に立たないように思えます。
しかし、3ビット反復符号を少し修正することで、位相反転エラーを検出できるようになります。
この修正を行うと、今度はビット反転エラーに対して無力になってしまいます。ですが、次のセクションで学ぶように、この修正済み符号と3ビット反復符号を組み合わせることで、ビット反転エラーと位相反転エラーの両方を修正できる「ショアの符号（Shor code）」を構築することができます。

以下に、これまで学んだ符号化回路を修正したバージョン を示します。この修正によって、位相反転誤りを検出できるようになります。
修正は非常に単純で、2つの制御NOT（CNOT）ゲートを適用した後、各量子ビットにアダマール（Hadamard）ゲートを適用するだけ です。

![Modified encoding circuit for the 3-bit repetition code](https://learning-api.quantum.ibm.com/assets/4129daaa-593c-4854-b96e-451c655ab30e?format=auto&quality=80)

アダマールゲートは、$\vert 0\rangle$ の状態を $\vert + \rangle$ の状態に変換し、$\vert 1\rangle$ の状態を $\vert - \rangle$ の状態に変換 します。したがって、その結果として、単一量子ビットの状態 $\alpha\vert 0\rangle + \beta \vert 1\rangle$ は、次のように符号化されます。

$$
\alpha \vert {+}\,{+}\,{+} \rangle + \beta \vert {-}\,{-}\,{-} \rangle
$$

ここで $\vert {+}\,{+}\,{+} \rangle = \vert + \rangle \otimes \vert + \rangle \otimes\vert + \rangle$ 、
$\vert {-}\,{-}\,{-} \rangle = \vert - \rangle \otimes \vert - \rangle \otimes\vert - \rangle$ を表ます。

位相反転エラー（または $Z$ ゲート） は、$\vert + \rangle$ と $\vert - \rangle$ の状態を入れ替える作用を持ちます。したがって、この符号化は位相反転エラーの検出（および修正）に有効 となります。
特に、これまでに学んだ誤り検出回路を以下のように修正することで、位相反転エラーを検出できるようになります。

![Phase error detection circuit for the 3-bit repetition code](https://learning-api.quantum.ibm.com/assets/69675bf2-0c26-4bd3-be6a-434fa26c8b4c?format=auto&quality=80)

言葉で説明すると、これまでの回路を使用し、単に最初と最後にアダマールゲートを上部の3つの量子ビットに適用するだけです。
このアイデアは、最初の3つのアダマールゲートが、$\vert + \rangle$ と $\vert - \rangle$ の状態を $\vert 0\rangle$ と $\vert 1\rangle$ の状態に戻す役割を果たし、その後、これまでと同じパリティチェックを実行し、最後のアダマールゲートの層によって、再び $\vert + \rangle$ と $\vert - \rangle$ の状態に戻すことで、元の符号化を復元するというものです。
今後の参考のために、この位相反転エラーの検出回路を以下のように簡略化できることを確認しておきます。

![Simplified phase error detection circuit](https://learning-api.quantum.ibm.com/assets/480ca2ea-ca94-4b9c-bc3c-e247f91e69d5?format=auto&quality=80)

以下の4つの回路図は、修正された3ビット反復符号（符号化ステップと誤り検出ステップを含む）が、最大1つの位相反転エラーが発生した場合にどのように機能するかを示しています。
その動作は、ビット反転に対する通常の3ビット反復符号と類似しています。

![Phase-flip error detection for modified 3-bit repetition code (no errors)](https://learning-api.quantum.ibm.com/assets/e10d2bbe-0e9d-4422-8565-2ba015de0114?format=auto&quality=80)

![Phase-flip error detection for modified 3-bit repetition code (no errors)](https://learning-api.quantum.ibm.com/assets/1a7744d4-fde3-49ff-8c53-8fd80fcbb24a?format=auto&quality=80)

![Phase-flip error detection for modified 3-bit repetition code (no errors)](https://learning-api.quantum.ibm.com/assets/b94d9da2-afee-4fe5-beb9-a8956dbfaa70?format=auto&quality=80)

![Phase-flip error detection for modified 3-bit repetition code (no errors)](https://learning-api.quantum.ibm.com/assets/f42d42e7-f8dc-455b-b376-b4e8b5880622?format=auto&quality=80)

上記の表と同様の表を、今回は最大1つの位相反転エラーが発生する可能性を考慮した場合について示します。

| 状態 | シンドローム | 修正 |
|:-:|:-:|:-:|
| $\alpha\vert {+}\,{+}\,{+} \rangle + \beta \vert {-}\,{-}\,{-}\rangle$ | $00$ | $\mathbb{I}\otimes\mathbb{I}\otimes\mathbb{I}$ |
| $\alpha\vert {-}\,{+}\,{+} \rangle + \beta \vert {+}\,{-}\,{-}\rangle$ | $10$ | $Z\otimes\mathbb{I}\otimes\mathbb{I}$ |
| $\alpha\vert {+}\,{-}\,{+}\rangle + \beta \vert {-}\,{+}\,{-}\rangle$ | $11$ | $\mathbb{I}\otimes Z\otimes\mathbb{I}$ |
| $\alpha\vert {+}\,{+}\,{-}\rangle + \beta \vert {-}\,{-}\,{+}\rangle$ | $01$ | $\mathbb{I}\otimes\mathbb{I}\otimes Z$ |

残念ながら、この修正された3ビット反復符号では、もはやビット反転エラーを訂正することはできません。
しかし、すべてが無駄になったわけではありません。
先に述べたように、これら2つの符号を組み合わせることで、9量子ビットのショアの符号を構成でき、それによってビット反転エラーと位相反転エラーの両方を訂正できるようになります（したがって、単一量子ビットに対するあらゆるエラーを訂正可能となります）。

## 9量子ビットのショアの符号

ここからは、9量子ビットのショアの符号 について説明します。この符号は、前のセクションで扱った2つの符号を組み合わせることで得られる量子誤り訂正符号です。つまり、単一のビット反転エラーを訂正できる3ビット反復符号と、単一の位相反転エラーを訂正できる修正版の3ビット反復符号を組み合わせることで構成されます。

### 符号の説明

正確には、9量子ビットのショアの符号は、前のセクションの2つの符号を *連結（concatenation）* することで得られる符号です。
これは、まず 1つの符号化を適用して1量子ビットを3量子ビットに符号化し、その後、最初の符号化で使用した3量子ビットの *それぞれ* に対して、もう一方の符号化を適用することで、最終的に9量子ビットとなる、という構造になっています。
今回のケースでは、2つの符号をどちらの順番で適用しても問題ありませんが、ここではまず位相反転エラーを検出できる修正版の3ビット反復符号を適用し、その後、得られた3つの量子ビットの *それぞれ* に対して、ビット反転エラーを検出できる元の3ビット反復符号を適用することにします。

以下に、この符号化を回路図で表したものを示します。

![Encoding circuit for the 9-qubit Shor code showing 3 blocks](https://learning-api.quantum.ibm.com/assets/1ba8abfe-352a-45d1-b6e2-8e2694868b6f?format=auto&quality=80)

この図が示すように、ショアの符号の9量子ビットは、3つのブロックに分けて考えることができます。各ブロックは、 *第2の* 符号化ステップ（通常の3ビット反復符号）によって得られます。  
ここで適用される通常の3ビット反復符号は、3回独立に適用されるため、 *内符号（inner code）* と呼ばれます。一方、第1の符号化ステップ（位相反転エラーを検出できる修正版の3ビット反復符号）は、 *外符号（outer code）* と呼ばれます。  

また、この符号を別の方法で表すこともできます。つまり、元の1量子ビットの2つの標準基底状態がどのように符号化されるかを記述する方法です。

$$
\begin{aligned}
\vert 0\rangle &
\:\mapsto\:
\frac{1}{2\sqrt{2}}
(\vert 000\rangle + \vert 111\rangle) \otimes
(\vert 000\rangle + \vert 111\rangle) \otimes
(\vert 000\rangle + \vert 111\rangle) \\[2.25mm]
\vert 1\rangle &
\:\mapsto\:
\frac{1}{2\sqrt{2}}
(\vert 000\rangle - \vert 111\rangle) \otimes
(\vert 000\rangle - \vert 111\rangle) \otimes
(\vert 000\rangle - \vert 111\rangle)
\end{aligned}
$$

これが分かれば、線形性を利用して任意の量子ビットの状態ベクトルがどのように符号化されるかを決定できます。

### エラーとCNOTゲート

エラーとCNOTゲートを分析するために、9量子ビットのショアの符号やその他の符号において、$X$ および $Z$ エラーが量子ビットの符号化にどのような影響を与えるかを理解するのに役立つ、いくつかの基本的な関係を確認しておくとよいでしょう。
9量子ビットのショアの符号の解析を始めるこのタイミングで、一旦立ち止まってこの点を整理するのは理にかなっています。  

以下の回路図は、$X$ ゲートとCNOTゲートの間に成り立つ3つの基本的な関係を示しています。
特に、 *ターゲット* 量子ビットに $X$ ゲートを適用してからCNOTを適用することは、順序を入れ替えて先にCNOTを適用することと等価です。
一方、 *コントロール* 量子ビットに $X$ ゲートを適用してからCNOTを適用することは、CNOTを適用した後に両方の量子ビットに $X$ ゲートを適用することと等価になります。
最後に、両方の量子ビットに $X$ ゲートを適用してからCNOTを適用することは、先にCNOTを適用し、その後コントロール量子ビットに $X$ ゲートを適用することと等価です。
これらの関係は、必要な行列の積を計算するか、回路が標準基底状態に与える影響を計算することで容易に確認できます。

![X errors before and after CNOT gates](https://learning-api.quantum.ibm.com/assets/7a9093d5-3a22-4442-9603-059f24af908b?format=auto&quality=80)

$Z$ ゲートの場合も同様の関係が成り立ちますが、コントロール量子ビットとターゲット量子ビットの役割が入れ替わります。
特に、次の量子回路で示される3つの関係が成り立ちます。

![Z errors before and after CNOT gates](https://learning-api.quantum.ibm.com/assets/f469c4e9-50b2-4a02-a345-29595ec61173?format=auto&quality=80)

### ビット反転エラーの訂正

次に、9量子ビットのショアの符号を使用してビット反転エラー（以後、便宜上 $X$エラーと呼びます）を検出および修正する方法を考えます。

$X$エラーを検出および修正するためには、符号化の3つのブロックをそれぞれ個別に扱うことができます。
各ブロックは3ビット反復コードを使用して量子ビットをエンコードしており、これにより$X$エラーから保護されています。
したがって、前のセクションで説明したシンドローム測定と$X$エラー修正を各ブロックに独立して行うことで、各ブロックごとに最大1つの$X$エラーを検出および修正できます。
特に、符号化の9量子ビットのうち最大1つの$X$エラーが発生した場合、この手順によって誤りは検出され、修正されます。

要するに、このコードでビット反転エラーを修正するのは簡単なことです。なぜなら、内符号がビット反転エラーを修正するからです。

### 位相反転エラーの訂正

次に、位相反転エラー、または便宜上 $Z$エラーを修正する方法を考えます。
今回は、少し異なり、どうすべきかが明確ではありません。というのも、外符号が$Z$誤りを検出する役割を果たす一方で、内符号が何らかの形で「邪魔」になっており、このエラーを検出して修正するのが少し難しくなっているからです。

ショアの符号の9量子ビットのうち1つに$Z$エラーが発生したと仮定しましょう。以下の図のように、例えばその1つの量子ビットで$Z$エラーが発生した場合です。

![Z error on a qubit for the 9-qubit Shor code](https://learning-api.quantum.ibm.com/assets/938e22ee-c84d-4995-886a-5a3935e58163?format=auto&quality=80)

既に、3ビット反復符号を使用しているときに$Z$エラーが発生した場合の影響を観察しています — これは、符号化前に$Z$エラーが発生したのと同じ効果を持ちます。
9量子ビットのショアの符号の文脈では、これは、3つのブロック内のいずれか1つの量子ビットで$Z$エラーが発生すると、その効果は常に同じであり、内符号が適用される前に対応する量子ビットで$Z$エラーが発生した場合と同じ効果になることを意味します。

例えば、上の回路図は以下の回路図と同じ効果を持ちます。
これは、上で説明した$Z$ゲートとCNOTゲートの関係を使用するか、単に任意の量子状態$\vert\psi\rangle$に対して回路を評価することで理由づけできます。

![Z error prior to the inner code for the 9-qubit Shor code](https://learning-api.quantum.ibm.com/assets/26fff6df-c61a-4946-a57a-c0bd09974add?format=auto&quality=80)

これにより、$Z$エラーを検出して訂正するための一つの方法が示唆されます。それは、内符号を *復号化* し、外側の符号化に使用される3つの量子ビットと6つの初期化されたワークスペース量子ビットを残す方法です。その後、外側の符号のこれらの3つの量子ビットを$Z$エラーについてチェックし、最終的に内符号を再符号化して、ショアの符号から得られる9量子ビットの符号化に戻します。もし$Z$エラーを検出した場合、内符号で再符号化する前に訂正するか、再符号化後にそのブロックの任意の量子ビットに$Z$ゲートを適用して訂正することができます。

こちらは、符号化回路と、上記で示したエラー、さらに先ほど説明した手順を含む回路図です（ただし、実際の訂正ステップは含まれていません）。

![Z error detection for the 9-qubit Shor code](https://learning-api.quantum.ibm.com/assets/beead3cf-92d0-441b-8fd0-81567fe01c8a?format=auto&quality=80)

この特定の例では、シンドローム測定の結果は $11$ となり、$Z$ エラーが中央のブロックのいずれかの量子ビットで発生したことが特定されます。  

$Z$ エラーの訂正を再符号化の前ではなく後に行うことの利点の一つは、上記の回路を簡略化できることです。以下の回路は等価ですが、CNOTゲートを4つ少なくできます。

![Simplified Z error detection for the 9-qubit Shor code](https://learning-api.quantum.ibm.com/assets/90849807-c5a9-4859-be0c-abe17277f813?format=auto&quality=80)

再び、シンドロームはどの量子ビットが $Z$ エラーの影響を受けたかを示すのではなく、どのブロックで $Z$ エラーが発生したかを示しており、ブロック内のどの量子ビットが影響を受けたとしても結果は同じです。
したがって、影響を受けたブロック内の3つの量子ビットのいずれか（具体的には、そのブロックの最上位の量子ビットとしましょう）に $Z$ ゲートを適用することでエラーを訂正できます。  

ちなみに、ここでは量子誤り訂正符号における *縮退性*（degeneracy）の例を見ることができます。つまり、特定のエラー（この場合は $Z$ エラー）を一意に特定できなくても訂正が可能であるという性質を示しています。

### 同時に発生するビット反転エラーと位相反転エラー

これまでに、9量子ビットのショアの符号を用いて $X$ エラーと $Z$ エラーをそれぞれ検出・訂正する方法を見てきました。特に、最大1つの $X$ エラーまたは最大1つの $Z$ エラーを検出・訂正できることを確認しました。
では、ビット反転エラーと位相反転エラーが同時に、しかも同じ量子ビットに発生した場合はどうなるでしょうか？
実は、この場合でも特別な対策を取る必要はありません。これまでに説明した方法のままで、最大1つの $X$ エラーと最大1つの $Z$ エラーを同時に検出・訂正することが可能です。  

具体的には、$X$ エラーは通常の3ビット反復符号によるシンドローム測定によって検出され、これは各3量子ビットのブロックごとに個別に行われます。
一方、$Z$ エラーは先ほど説明した方法で検出されます。これは、内符号をデコードし、位相反転エラーを検出するための修正3ビット反復符号のシンドローム測定を行い、その後再び内符号を符号化する処理と等価です。
これら2つの誤り検出手順（およびそれに対応する訂正手順）は完全に独立して実行することができ、実際のところ、どちらを先に実行しても問題はありません。  

この仕組みを理解するために、次の回路図に示される例を考えます。この例では、中央のブロックの最下位の量子ビットに $X$ エラーと $Z$ エラーの両方が発生しています。

![An XZ error for the 9-qubit Shor code](https://learning-api.quantum.ibm.com/assets/a6944302-fc87-474e-ad3c-d8768694c70e?format=auto&quality=80)

まず、誤りの順序が問題にならないことを確認しましょう。つまり、$X$ エラーと $Z$ エラーの位置を入れ替えたとしても、同等な回路が得られるということです。
ただし、誤解のないように言うと、$X$ と $Z$ は可換ではなく、*反交換* であることに注意してください。

$$
XZ =
\begin{pmatrix}
0 & 1\\[1mm]
1 & 0
\end{pmatrix}
\begin{pmatrix}
1 & 0\\[1mm]
0 & -1
\end{pmatrix}
= \begin{pmatrix}
0 & -1\\[1mm]
1 & 0
\end{pmatrix}
= - \begin{pmatrix}
1 & 0\\[1mm]
0 & -1
\end{pmatrix}
\begin{pmatrix}
0 & 1\\[1mm]
1 & 0
\end{pmatrix}
= -ZX.
$$

これは、以下の回路が上の回路とグローバル位相因子 $-1$ を除いて等価であることを意味します。

![A ZX error for the 9-qubit Shor code](https://learning-api.quantum.ibm.com/assets/f40338ac-9c4c-4eeb-84a0-7ed85e47c679?format=auto&quality=80)

これまでと同様に、$Z$ エラーを移動させることで、別の等価な回路を得ることができます。

![A ZX error for the 9-qubit Shor code](https://learning-api.quantum.ibm.com/assets/d26eebb6-94e9-467e-a5e8-4d71685c528f?format=auto&quality=80)

この時点で明らかなのは、まず $X$ エラーを検出・修正する手順を実行すれば、$X$ エラーが修正され、その後に $Z$ エラーを検出・修正する手順を実行すれば、これまでと同様に $Z$ エラーも取り除くことができるということです。  

また、$Z$ エラーを検出・修正する手順を先に実行することも可能です。
この手順が、1つ以上の $X$ エラーが存在する場合でも正しく機能することは、符号化に使用される9つの量子ビットの任意の $X$ ゲートが、$Z$ エラーのシンドロームを測定するための簡略化された回路のすべてのゲートと可換であるという事実から導かれます。
したがって、このシンドローム測定は、どのブロックに $Z$ エラーが発生したかを正しく特定できます。
また、あるブロックに $Z$ エラーが発生した場合に、そのブロック内の任意の量子ビットに $Z$ ゲートを適用することでエラーが修正されることは、$X$エラーが起こったとしても、$X$ と $Z$ ゲートの順序を変更した場合にグローバルな位相因子を除けば等価な回路になるという先ほどの議論と同じ理由から成り立ちます。

したがって、9量子ビットのショアの符号は、符号化に使用される9つの量子ビットのいずれかに生じた $X$ エラー、$Z$ エラー、またはその両方を修正することができます。さらに、この符号はより多くのエラーも修正可能です。
例えば、異なるブロックに分かれた複数の $X$ エラーや、最大でも1つのブロックに奇数個発生した $Z$ エラーであれば修正できます。
しかし、今後の議論において重要なのは、任意の1つの量子ビットに対する $X$ エラー、$Z$ エラー、またはその両方を修正できるという点です。

### ランダムなエラー

本講義の最後のセクションでは任意の量子エラーについて考察しますが、その前に、パウリ行列で表されるエラーが *ランダムに* 量子ビットに発生する場合の9量子ビットのショアの符号の性能について簡単に考えてみましょう。

具体的には、エラーが各量子ビットに対して *独立に* 発生し、それぞれの量子ビットが確率 $p$ でエラーを経験すると仮定します。また、異なる量子ビット間のエラーには相関がないとします。これは、古典ビットに対する二元対称通信路（binary symmetric channel）と類似したノイズモデルです。
$X, Y, Z$ のエラーが発生する確率をそれぞれ異なる値にすることもできますが、できるだけ簡単にするために、ここでは9量子ビットのショアの符号にとって最悪のケースを考えます。つまり、エラーが発生した各量子ビットに $Y$ エラーが生じる場合を想定します。
ここで、$Y$ エラーとは（無視できるグローバル位相を除けば）同じ量子ビットに $X$ エラーと $Z$ エラーが同時に発生することと等価です。これは、$Y = iXZ$ という関係式からもわかります。このため、これまで $Y$ エラーについて特に考慮してこなかったように見えますが、実際には $X$ および $Z$ エラーの組み合わせとして含まれていたのです。

さて、$\mathsf{Q}$ をエラーから保護したいある特定の状態にある量子ビットとします。ここで、9量子ビットのショアの符号を使用するという選択肢を考えたとき、「この符号を使うべきか？」という自然な疑問が生じます。

この答えは、必ずしも「はい」ではありません。
ノイズが多すぎる、つまりこの場合 $p$ が大きすぎると、ショアの符号を使用することが逆に悪影響を及ぼす可能性があります。これは、$p$ が 1/2 を超えると3ビット反復符号（3-bit repetition code）が使わない場合よりも性能が悪くなるのと同様です。
しかし、$p$ が十分に小さい場合は「はい」、この符号を使うべきです。なぜなら、この符号を使うことで符号化された状態が破損する確率を減らすことができるからです。
では、これがなぜ成り立つのか、またこの符号にとって $p$ が大きすぎる場合や十分に小さい場合とはどのような条件なのかを考えてみましょう。

ショアの符号は、単一の量子ビットに発生した任意のパウリエラーを修正できます。もちろん $Y$ エラーも含まれます。しかし、2つ以上の量子ビットに $Y$ エラーが発生した場合には、適切に修正することができません。
ここで重要なのは、我々が先ほど説明した $X$ および $Z$ エラー修正の手順を用いることを前提としている点です。
もし、事前に $Y$ エラーのみを考慮すればよいとわかっていれば、当然ながら異なる修正手順を選択するでしょう。
しかし、それはノイズモデルを不正に操作することになり、異なるパウリエラーを選択すれば、どのような修正手順も2つ以上の量子ビットにエラーが発生した場合には失敗するように常にモデルを設定できてしまいます。

したがって、この符号が $\mathsf{Q}$ を保護できるのは、符号化された9つの量子ビットのうち高々1つにしかエラーが発生しない場合です。
その確率は以下の通りです。

$$
(1-p)^9 + 9 p (1-p)^8.
$$

そうでない場合、すなわち確率

$$
1 - (1-p)^9 - 9 p (1-p)^8,
$$

で、この符号は $\mathsf{Q}$ を保護できません。

ここで、「保護できない」とは具体的にどういうことかというと、グローバル位相を除けば、符号化された論理量子ビット $\mathsf{Q}$ に単位行列でないパウリ演算が適用されるということです。
つまり、これまで説明してきたショアの符号の $X$ および $Z$ エラー検出・修正を実行した結果、最終的に得られる状態は、元の状態 $\mathsf{Q}$ に単位行列でないパウリ演算を適用したものと等価（グローバル位相を除く）な符号化状態になります。
これを簡単に言い換えると、 *論理* エラーが発生した、ということになります。
この論理エラーが、もともとの量子ビット $\mathsf{Q}$ の状態（すなわち、9個の *物理量子ビット* を使って符号化した *論理量子ビット*）に影響を及ぼすかどうかは場合によりますが、本解析ではこの事象が発生した時点で「符号の失敗」と見なします。

一方、符号を使わなかった場合、唯一の量子ビットは確率 $p$ で同様に単位行列でないパウリ演算を受けることになります。
したがって、このコードは、第一の確率が第二の確率より小さい場合に有効です：

$$
1 - (1-p)^9 - 9 p (1-p)^8 < p.
$$

以下のプロットは、非常に小さい $p$ の値に対して符号が有利に働くことを示しており、損益分岐点はおよそ $0.0323$ の地点で発生することを示しています。

![Error probability graph for independent Y errors using the Shor code](https://learning-api.quantum.ibm.com/assets/6ca725ac-d090-449d-b6db-2bf586139ac6?format=auto&quality=80)

もし $p$ がこの損益分岐点より小さい場合、コードは有利に働きます；
損益分岐点では確率が等しくなり、コードを使用すると8つの量子ビットと共に時間を無駄にしていることになります；
そして損益分岐点を越えると、このコードは $\mathsf{Q}$ における論理的エラーの確率を *増加* させるため、絶対に使用すべきではありません。

約3.25%というのは、あまり良い損益分岐点には思えないかもしれません。特に、これは古典情報のための3ビット繰り返しコードの損益分岐点である $50\%$ と比較すると明らかです。
この違いは、大きな理由は、量子情報が古典情報よりもデリケートで保護するのが難しいためです。
また、9量子ビットのショアの符号が世界初の量子誤り訂正コードとして素晴らしい発見であることを認めつつも、実際的な観点ではあまり良い符号ではないことも認識するべきです。

## エラーの離散化

これまで、9量子ビットのショアの符号の文脈で $X$ エラーと $Z$ エラーについて考えてきました。
このセクションでは、任意のエラーについて考察します。そして、わかることは、そのようなエラーを処理するために、実際にはこれまでに議論したこととは何も異なることをする必要はないということです；
$X$ エラー、$Z$ エラー、またはその両方を訂正する能力は、任意のエラーを訂正する能力を意味します。
この現象は、時に *エラーの離散化* と呼ばれます。

### ユニタリー量子ビットエラー

単一量子ビットの *ユニタリー* エラーについて考えることから始めましょう。
例えば、このようなエラーは、ブロッホ球面の非常に小さな回転に対応するかもしれません。これは、ゲートが完璧でないことによるエラーを表すことがあります。
または、それは単一量子ビットに対する他のユニタリー操作であり、必ずしも単位行列に近いものではありません。
このようなエラーを訂正するのは難しいように思えるかもしれません。
結局のところ、このようなエラーは無限に多く存在し、エラーが何であるかを正確に特定して、それを元に戻すことができるとは考えにくいからです。

しかし、ビット反転、位相反転、またはその両方を訂正できるならば、実際にはこのレッスンで説明した手順を使用することで、任意の単一量子ビットのユニタリーエラーを訂正することに成功することがわかります。
その理由は、任意の $2 \times 2$ ユニタリー行列 $U$（単一量子ビットのエラーを表す）を、4つのパウリ行列（単位行列を含む）の線形結合として表現できるという認識から始まります。

$$
U = \alpha \mathbb{I} + \beta X + \gamma Y + \delta Z
$$

これから分かるように、エラー検出回路を実行すると、私たちにシンドロームビットを与える測定が、確率的に符号化された状態を、4つのパウリ行列のいずれかによって表されるエラー（またはエラーがない状態）に収縮させることになります。
（上記の線形結合がユニタリであるための必要条件ではありますが十分ではない条件として、$\alpha$、$\beta$、$\gamma$、$\delta$ に対して、$\vert\alpha\vert^2 + \vert\beta\vert^2 + \vert\gamma\vert^2 + \vert\delta\vert^2 = 1$ であることが挙げられます。実際、$\vert\alpha\vert^2$、$\vert\beta\vert^2$、$\vert\gamma\vert^2$、$\vert\delta\vert^2$ は、符号化された状態が対応するパウリエラーが発生した状態に収縮する確率です。）

この仕組みがどのように機能するかを詳しく説明するために、サブインデックスを使用して、特定の単一量子ビットのユニタリー操作がどの量子ビットに作用するかを示すのが便利です。
例えば、Qiskitの量子ビット番号付け規約 $(\mathsf{Q}_8, \mathsf{Q}_7, \ldots, \mathsf{Q}_0)$ を使用して、ショアの符号に使用される9つの量子ビットに番号を付けると、次のように各単一量子ビットに対するユニタリー操作の式を表すことができます。この場合、各ユニタリー行列は他の量子ビット上で単位行列とテンソル積を取ります。

$$
\begin{aligned}
X_0 & = \mathbb{I} \otimes \mathbb{I} \otimes \mathbb{I} \otimes \mathbb{I} \otimes \mathbb{I} \otimes \mathbb{I} \otimes \mathbb{I} \otimes \mathbb{I} \otimes X \\[1.5mm]
Z_4 & = \mathbb{I} \otimes \mathbb{I} \otimes \mathbb{I} \otimes \mathbb{I} \otimes Z \otimes \mathbb{I} \otimes \mathbb{I} \otimes \mathbb{I} \otimes \mathbb{I} \\[1.5mm]
U_7 & = \mathbb{I} \otimes U \otimes \mathbb{I} \otimes \mathbb{I} \otimes \mathbb{I} \otimes \mathbb{I} \otimes \mathbb{I} \otimes \mathbb{I} \otimes \mathbb{I}
\end{aligned}
$$

したがって、特に、与えられた量子ビットユニタリー演算子 $U$ に対して、$U$ が量子ビット $k$ に適用される作用を、次の式で指定できます。これは、前述の式と似ていますが、各行列が量子ビット $k$ に適用される操作を表す点が異なります。

$$
U_k = \alpha \mathbb{I}_k + \beta X_k + \gamma Y_k + \delta Z_k
$$

今、$\vert\psi\rangle$ が量子ビットの状態の9量子ビットの符号化であると仮定します。
もしエラー $U$ が量子ビット $k$ に発生した場合、状態 $U_k \vert\psi\rangle$ が得られ、この状態は次のようにして、$\vert\psi\rangle$ に作用するパウリ演算の線形結合として表すことができます。

$$
U_k \vert\psi\rangle = \alpha \vert\psi\rangle + \beta X_k\vert\psi\rangle + \gamma Y_k\vert\psi\rangle + \delta Z_k\vert\psi\rangle
$$

ここで、$Y = iXZ$ の置換を行いましょう。

$$
U_k \vert\psi\rangle = \alpha \vert\psi\rangle + \beta X_k\vert\psi\rangle + i \gamma X_kZ_k\vert\psi\rangle + \delta Z_k\vert\psi\rangle
$$

ここで、前に説明した誤り検出と修正の手順を考えましょう。
3つの内符号のパリティーチェックと外符号のパリティーチェックに対する測定結果を合わせて、8ビットからなる1つのシンドロームとして考えることができます。
これらのシンドロームビットを生成する標準基底測定の直前で、状態は次のような形になります。

$$
\alpha\,\vert\text{$\mathbb{I}$ syndrome}\rangle \otimes \vert\psi\rangle
+ \beta\,\vert\text{$X_k$ syndrome}\rangle \otimes X_k\vert\psi\rangle
+ i \gamma\,\vert\text{$X_k Z_k$ syndrome}\rangle \otimes X_k Z_k\vert\psi\rangle
+ \delta\,\vert\text{$Z_k$ syndrome}\rangle \otimes Z_k\vert\psi\rangle
$$

ここで明確にするために、現時点で2つのシステムがあります。
左側のシステムはシンドロームを取得するために測定する8つの量子ビットで、$\vert \mathbb{I} \text{ syndrome}\rangle,$
$\vert X_k \text{ syndrome}\rangle,$ などは対応するエラー（または、$\mathbb{I}$ の場合はエラーではない）と一致する8ビットの標準基底状態を指します。
右側のシステムは符号化に使用する9つの量子ビットです。  

これらの2つのシステムは現在（一般的に）相関していることに注意してください。これがこの方法がうまく機能する鍵です。  
シンドロームを測定することによって、右側の9つの量子ビットの状態は、測定されたシンドロームと一致するパウリエラーが1つの量子ビットに適用された状態に実質的に収縮します。  
さらに、シンドローム自体は、エラーを元に戻して元の符号化された$\vert\psi\rangle$を回復できるだけの十分な情報を提供します。  
具体的には、シンドローム量子ビットが測定され、適切な修正が行われると、次のような形の状態が得られます。

$$
\xi \otimes \vert\psi\rangle\langle\psi\vert
$$

ここで

$$
\begin{aligned}
\xi = & \vert\alpha\vert^2 \vert\text{$\mathbb{I}$ syndrome}\rangle\langle\text{$\mathbb{I}$ syndrome}\vert \\[1mm]
& + \vert\beta\vert^2 \vert\text{$X_k$ syndrome}\rangle\langle\text{$X_k$ syndrome}\vert\\[1mm]
& + \vert\gamma\vert^2 \vert\text{$X_k Z_k$ syndrome}\rangle\langle\text{$X_k Z_k$ syndrome}\vert\\[1mm]
& + \vert\delta\vert^2 \vert\text{$Z_k$ syndrome}\rangle\langle\text{$Z_k$ syndrome}\vert.
\end{aligned}
$$

重要なのは、これは積状態であるということです：
右側のテンソル因子には元の、非収縮の符号化の状態があり、左側にはランダムな誤りシンドロームを記述する密度行列$\xi$があります。  
右側のシステムとはもはや相関がなく、私たちが関心を持っているのは右側のシステムだからです。誤りは修正されたため、シンドローム量子ビットは捨てるか、再利用できるようにリセットすることができます。
これが、誤りによって生成されたランダム性、または *エントロピー* がシステムから除去される方法です。

これは、ユニタリーエラーの特別な場合におけるエラーの離散化です。  
本質的に、シンドロームを測定することによって、エラーをパウリ行列で記述されたエラーに効果的に *射影* しています。

一見すると、このように任意のユニタリーエラー、たとえ非常に小さくてほとんど気づかれないエラーであっても修正できるというのは、あまりにも良すぎるように思えるかもしれません。
しかし、ここで重要なのは、これは *単一* の量子ビットに対するユニタリーエラーであり、コードの設計によって、単一の量子ビット操作が符号化された論理量子ビットの状態を変更できないということです。  
それができることは、有効な符号化のサブスペースから状態を移動させることだけですが、その場合、誤り検出によって状態が収縮し、修正でそれを元の位置に戻します。

### 任意の量子ビットエラー

最後に、任意のユニタリーエラーでない誤りを考えましょう。 
正確には、任意の量子ビットチャネル$\Phi$によって記述される誤りを考えます。
例えば、これは位相減衰チャネルや脱分極チャネル、リセットチャネル、またはこれまで考えたことのないような奇妙なチャネルである可能性があります。

最初のステップは、$\Phi$の任意のクラウス表現を考えることです。

$$
\Phi(\sigma) = \sum_j A_j \sigma A_j^{\dagger}
$$

これは量子ビットのチャネルなので、各$A_j$は$2 \times 2$行列であり、これをパウリ行列の線形結合として表現できます。

$$
A_j = \alpha_j \mathbb{I} + \beta_j X + \gamma_j Y + \delta_j Z
$$

これにより、エラー$\Phi$が選ばれた量子ビット$k$に対して与える作用を、パウリ行列を用いて次のように表現できます。

$$
\Phi_k \bigl( \vert\psi\rangle\langle\psi\vert\bigr) =
\sum_j (\alpha_j \mathbb{I}_k + \beta_j X_k + \gamma_j Y_k + \delta_j Z_k) \vert\psi\rangle\langle\psi\vert
(\alpha_j \mathbb{I}_k + \beta_j X_k + \gamma_j Y_k + \delta_j Z_k)^{\dagger}
$$

要するに、私たちはすべてのクラウス行列をパウリ行列の線形結合として展開しただけです。

もし今、エラーのシンドロームを計算して測定し、明らかになったエラーを修正すれば、ユニタリーエラーの場合と同様の状態が得られます。

$$
\xi \otimes \vert\psi\rangle\langle\psi\vert
$$

今回は次のようになります。

$$
\begin{aligned}
\xi = & \sum_j \Bigl(\vert\alpha_j\vert^2 \vert\text{$\mathbb{I}$ syndrome}\rangle\langle\text{$\mathbb{I}$ syndrome}\vert \\[-3mm]
& \qquad + \vert\beta_j\vert^2 \vert\text{$X_k$ syndrome}\rangle\langle\text{$X_k$ syndrome}\vert\\[2mm]
& \qquad + \vert\gamma_j\vert^2 \vert\text{$X_k Z_k$ syndrome}\rangle\langle\text{$X_k Z_k$ syndrome}\vert\\[2mm]
& \qquad + \vert\delta_j\vert^2 \vert\text{$Z_k$ syndrome}\rangle\langle\text{$Z_k$ syndrome}\vert \Bigr).
\end{aligned}
$$

詳細は少し複雑で、ここでは示されていませんが、概念的にはユニタリーの場合と同じです。

### 一般化

エラーの離散化は、複数の量子ビットに対するエラーを検出・修正できる他の量子誤り訂正符号にも一般化できます。この場合、複数の量子ビットに対するエラーは、パウリ行列の *テンソル積* として表現でき、対応する異なるシンドロームは、複数の量子ビットに対して行われる可能性のあるパウリ操作の修正を指定します。

再度言うと、シンドロームを測定することによって、エラーは効果的にパウリ行列のテンソル積によって表される離散的な可能性のセットに射影または収束し、それらのパウリエラーを修正することによって、元の符号化された状態を回復できます。
一方、この過程で生成されたランダムネスはシンドローム量子ビットに移され、これらは破棄またはリセットされるため、符号化を保持しているシステムからこの過程で生成されたランダムネスが除去されます。